In [31]:
%%writefile data_generator.py
#preprocess the data & generate vocabulary
import numpy as np
import csv
import operator
import copy
import os
import random
import gensim
import pickle
from data_processor import *
np.random.seed(1)


def extract_data(input_name,output_name,cols= [],processor=[],criteria = (lambda row:True),delimiter = "\t"):
    '''
    extract data from a Unstructured Stream File Generated By Cosmos.
    default delimiter is \t
    Processor process each row before output
    and criteria decide whether the row is in output
    '''
    data=open(input_name,"r",encoding='utf_8')
    output=open(output_name,"w",encoding='utf_8')
    
    if len(cols) == 0:
        print("Empty Output.")
        return
    if (len(cols) != len(processor)) and (len(processor) != 0):
        print("Wrong Number of input,or processor.")
        return
    count=0
    for line in data.readlines():
        row = line.rstrip("\n").split(delimiter)
        if not criteria(row):
            continue
          
        for i,index in enumerate(cols):
            if len(processor):
                output.write(processor[i](row[index]))
            else:
                output.write(row[index])
            output.write("\n") if i == len(cols)-1 else output.write('\t')
        count+=1
        if count%50000==0:
            print(count)


    data.close()
    output.close()
    print("Finished Extraction, Rows:{}, Cols:{}.".format(count,len(cols)))
    
def csv2csv(input_file,output_file,input_delimiter = "\t",output_delimiter=","):
    output = open(output_file,"w",encoding = "utf-8")
    
    for line in open(input_file,encoding="utf-8").readlines():
        output.write(output_delimiter.join(line.split(input_delimiter)))
        
    output.close()
    
    print("Generate New CSV File Successfully.")
    

def shuffle_data(data_file,out_data_file=None):
    data  = open(data_file,encoding = "utf-8").readlines()
    random.shuffle(data)
    if out_data_file:
        out_data = open(out_data_file,"w",encoding="utf-8")
    else:
        out_data = open(data_file,"w",encoding="utf-8")
    for line in data:
        out_data.write(line)
    out_data.close()
    print("Data Shuffle Succeed.")

def tokenize_data(input_file,output_file,no_token_list =[],filters ={}):
    '''
    need to import process
    '''
    print("Generating Tokenized Data ...")
    
    data=open(input_file,"r",encoding='utf8')
    output=open(output_file,"w",encoding='utf_8')
    #dict_list = [{} for x in range(len(data.readline().split("\t")))] 
    count =0
    for line in data.readlines():
        row = line.rstrip("\n").split("\t")
        for col in range(len(row)):
            if col not in no_token_list:
                row[col] = tokenize(row[col])
        for col in filters.keys():
            if filters[col](row[col]):
                continue
        #for col in range(len(row)):
        #    _make_dict(row[col],dict_list[col])
                    
        out = '\t'.join(row)
        output.write(out+"\n")
        count+=1
        if count%50000 == 0:
            print(count)
    output.close()
    data.close()
    print("Finished Generating Tokenized Data. Total:{} .".format(count))
    #return dict_list
def _make_dict(string,dic,tokenized=True):
    if not tokenized:
        tokens=[string]
    else:
        tokens=string.split(" ")
    for x in tokens:
        if x in dic:
            dic[x]+=1
        else:
            dic[x]=1
def dict_data(input_file,no_token_list =[]):
    
    
    data=open(input_file,"r",encoding='utf8')
    dict_list = [{} for x in range(len(data.readline().split("\t")))] 
    for line in data.readlines():
        row = line.rstrip("\n").split("\t")
        for col in range(len(row)):
            if col in no_token_list:
                _make_dict(row[col],dict_list[col],tokenized=False)
            else:
                _make_dict(row[col],dict_list[col],tokenized=True)
    return dict_list

def dict2file(dict_list,dict_file_list,criteria,with_count=False):
    for i in range(len(dict_file_list)):
        sorted_dict = sorted(dict_list[i].items(), key=operator.itemgetter(1))
        file = open(dict_file_list[i],"w",encoding="utf-8")
        for x in sorted_dict:
            
            if with_count and criteria[i](x[1]):
                file.write(x[0]+"\t"+str(x[1])+"\n")
            elif criteria[i](x[1]):
                file.write(x[0]+"\n")
        if len(sorted_dict)>200 and not with_count:
            file.write("UNK\n")
        file.close()
        
def sample_data(input_file,output_file,num_samples_per_class,per_class=1,classifier=(lambda x:0)):
    data = open(input_file,encoding = "utf-8").readlines()
    random.shuffle(data)
    output = open(output_file,"w",encoding = "utf-8")
    class_count = {x:0 for x in range(per_class)}
    
    for line in data:
        if class_count[classifier(line)]<num_samples_per_class:
            class_count[classifier(line)]+=1
            output.write(line)
    print("Sample Data Succeed. Total: {}".format(num_samples_per_class*per_class))   
        
def split_data(input_file,train_file,test_file,split_count=0,industry_dict=None):
    '''
    split count can either be a integer or a fraction(percentage)
    '''
    data = open(input_file,"r",encoding='utf_8').readlines()
    train = open(train_file,"w",encoding='utf_8')
    test = open(test_file,"w",encoding='utf_8')
    if industry_dict:
        temp_indusrey_dict = copy.copy(industry_dict)
        print(industry_dict.keys())
        for x in industry_dict.keys():        
            temp_industry_dict[x]=int(industry_dict[x]*0.7)
    if split_count == 0:
        split_count = int(len(data)*0.7)
    elif 0 < split_count < 1 :
        split_count = int(len(data)*split_count)
    if industry_dict:
        for line in data:
            industry = line.rstrip("\n").split("\t")[1]
            if temp_industry_dict[industry]==0:
                test.write(line)
            else:
                temp_industry_dict[industry]-=1
                train.write(line)
    else:
        train_data=data[:split_count]
        test_data = data[split_count:]
        
        for line in train_data:
            train.write(line)
        
        for line in test_data:
            test.write(line)
    train.close()
    test.close()
    print("Split Succeed.Train:{} Test:{}".format(split_count,len(data)-split_count))
    
    
def ctf_data(input_name,output_name,dict_name_list):
    '''
    generating ctf data the number of dict should be more than zero 
    IMPORTANT: 
        it should be customized on your computer/server if you want to run properly
    '''
    py_root = "C:\Local\Anaconda3-4.1.1-Windows-x86_64\envs\cntk-py35\python.exe"
    py_file = "C:/Local/CNTK-2.5.1/cntk/Scripts/txt2ctf.py"
    if len(dict_name_list) == 0:
        print("Empty dict list.")
        return
    command = " {} {} --map {} --annotated True --input {} --output {} --unk UNK".format(py_root,py_file," ".join(dict_name_list),input_name,output_name)
    #print(command)
    value = os.system(command)
    if value == 0:
        print("Finished Generating CTF Data.")
    else:
        print("Error in Executing Command.")
        print(command)
        
def union_data(input_files,output_file,shuffle=True):
    output = open(output_file,"w",encoding = "utf-8")
    for file in input_files:
        for line in open(file,encoding = "utf-8").readlines():
            output.write_line()
    output.close()
    if shuffle:
        shuffle_data(output_file)
        
def dedup_data(input_file,output_file = None,selector = lambda row:row):
    data = open(input_file,encoding = "utf-8").readlines()
    row_dict = {}
    
    for line in data:
        row_dict[selector(line)] = line
    #data.close()
    if output_file:
        output = open(output_file,"w",encoding = "utf-8")
    else:
        output = open(input_file,"w",encoding = "utf-8")
    for key in row_dict.keys():
        output.write(row_dict[key])
    output.close()
    print("Dedup Successfully. Before:{} After:{}".format(len(data),len(row_dict)))
    
def embed_dict(w2v_dict,embed_file,vocab_dict):

    model = gensim.models.KeyedVectors.load_word2vec_format(w2v_dict, binary=True)
    sorted_dict = sorted(vocab_dict.items(), key=operator.itemgetter(1))
    w2v_emb = []
    count = 0
    for pair in sorted_dict:
                word = pair[0]
                if word == "UNK":
                    #print("o")
                    w2v_emb.append(np.random.normal(0,1/300,300))

                else:
                    try:
                        w2v_emb.append(model[word])
                        count+=1
                    except:
                        #print(word)
                        w2v_emb.append(np.random.normal(0,1/300,300))
                #else:
                #    w2v_dict.append(np.zeros((300)))
    with open(embed_file, 'wb') as handle:
                    pickle.dump(np.array(w2v_emb), handle)
    print("Embedding File Generate Successfully. Total: {} Embedded: {}".format(len(w2v_emb),count))


Overwriting data_generator.py
